In [1]:
import torch as tch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, precision_recall_curve
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import joblib

In [2]:
device = tch.device("cuda" if tch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
# -------------------------------
# LSTM Model
# -------------------------------
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(NeuralNet, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.3)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        h0 = tch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = tch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]
        out = self.fc(out)
        return out


# -------------------------------
# Focal Loss for imbalanced data
# -------------------------------
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        BCE = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        probs = tch.sigmoid(inputs)
        pt = tch.where(targets == 1, probs, 1 - probs)
        loss = self.alpha * (1 - pt) ** self.gamma * BCE
        return loss.mean()


# -------------------------------
# Sequence creation
# -------------------------------
def create_sequences(data, labels, seq_len, step=10):
    X, y = [], []
    for i in range(0, len(data) - seq_len, step):  # step instead of 1
        X.append(data[i:i+seq_len])
        y.append(labels[i+seq_len])
    return np.array(X), np.array(y)

In [4]:
# -------------------------------
# Dataset class
# -------------------------------
class ESADataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = tch.tensor(self.data[idx]).float()
        y = tch.tensor(self.labels[idx]).float().unsqueeze(-1)
        return x, y

In [5]:
# -------------------------------
# Train model
# -------------------------------
def train_model(model, optimizer, scheduler, criterion, start_epoch, num_epoch, train_loader):
    for epoch in range(start_epoch, num_epoch):
        model.train()
        epoch_loss = 0.0
        grad_norm_sum = 0.0
        grad_batches = 0

        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()

            # Gradient clipping
            tch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=3.0)
            optimizer.step()

            total_norm = 0.0
            for p in model.parameters():
                if p.grad is not None:
                    param_norm = p.grad.data.norm(2)
                    total_norm += param_norm.item() ** 2
            total_norm = total_norm ** 0.5
            grad_norm_sum += total_norm
            grad_batches += 1

            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(train_loader)
        avg_grad_norm = grad_norm_sum / grad_batches
        lr = optimizer.param_groups[0]['lr']

        print(f"Epoch {epoch+1}/{num_epoch}, Loss: {avg_loss:.4f}, Avg Grad Norm: {avg_grad_norm:.4f}, LR: {lr:.6f}")

        if isinstance(scheduler, tch.optim.lr_scheduler.ReduceLROnPlateau):
            scheduler.step(avg_loss)
        else:
            scheduler.step()

        # Save checkpoint
        tch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'loss': avg_loss,
        }, f'models/Epoch_save/checkpoint_epoch{epoch}.pth')

In [26]:

# -------------------------------
# DATA PREPARATION
# -------------------------------
dataset = pd.read_csv("data/sample_dataset.csv")
dataset['datetime'] = pd.to_datetime(dataset['datetime'], utc=True)
dataset = dataset.set_index('datetime')

features = dataset.drop(columns=['label', 'Tc_lvl2', 'Tc_lvl3'])
labels = dataset['label']

# Resample to 4 minutes
features_resampled = features.resample('240s').ffill()
labels_resampled = labels.resample('240s').max().reindex(features_resampled.index)

resampled_df = features_resampled.join(labels_resampled)
resampled_df.fillna(0, inplace=True)

# Feature Engineering
for col in features_resampled.columns:
    resampled_df[f"{col}_diff"] = resampled_df[col].diff()
    resampled_df[f"{col}_std"] = resampled_df[col].rolling(30).std()

resampled_df.fillna(0, inplace=True)

# Prepare arrays
X_all = resampled_df.drop(columns=["label"]).to_numpy()
y_all = resampled_df["label"].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.3, random_state=42, shuffle=False)


In [27]:
#Scaling dataset
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
joblib.dump(scaler, 'models/Scalar_save/scaler.pkl')

['models/Scalar_save/scaler.pkl']

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
# Sequence creation
seq_length = 120
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train, seq_length)

# Compute weights for sampler
y_train_seq_t = tch.from_numpy(y_train_seq).float()
class_sample_count = np.array([(y_train_seq_t == 0).sum().item(), (y_train_seq_t == 1).sum().item()])
weights = 1. / class_sample_count
samples_weight = np.array([weights[int(t)] for t in y_train_seq])
samples_weight = tch.from_numpy(samples_weight)
sampler = WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight), replacement=True)

In [18]:
# Dataset + Dataloader
train_dataset = ESADataset(X_train_seq, y_train_seq)
train_loader = DataLoader(train_dataset, batch_size=512, sampler=sampler, num_workers=2)

In [19]:
#Model parameters
n_features = X_train_seq.shape[2]
hidden_size = 100
num_layers = 3
num_epoch = 50

In [20]:
# Model creation with optimizer and learning scheduler
model = NeuralNet(n_features, hidden_size, num_layers).to(device)
optimizer = tch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = tch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)
criterion = FocalLoss(alpha=0.25, gamma=2.0).to(device)

In [29]:
import os

# Create the directory for saving model checkpoints if it doesn't exist
os.makedirs('models/Epoch_save', exist_ok=True)

# -------------------------------
# MODEL TRAINING
# -------------------------------
train_model(model, optimizer, scheduler, criterion, start_epoch=0, num_epoch=num_epoch, train_loader=train_loader)

Epoch 1/50, Loss: 0.0432, Avg Grad Norm: 0.0073, LR: 0.000100
Epoch 2/50, Loss: 0.0432, Avg Grad Norm: 0.0071, LR: 0.000100
Epoch 3/50, Loss: 0.0433, Avg Grad Norm: 0.0014, LR: 0.000100
Epoch 4/50, Loss: 0.0430, Avg Grad Norm: 0.0131, LR: 0.000100
Epoch 5/50, Loss: 0.0435, Avg Grad Norm: 0.0117, LR: 0.000100
Epoch 6/50, Loss: 0.0432, Avg Grad Norm: 0.0038, LR: 0.000100
Epoch 7/50, Loss: 0.0433, Avg Grad Norm: 0.0011, LR: 0.000100
Epoch 8/50, Loss: 0.0437, Avg Grad Norm: 0.0211, LR: 0.000100
Epoch 9/50, Loss: 0.0429, Avg Grad Norm: 0.0191, LR: 0.000010
Epoch 10/50, Loss: 0.0430, Avg Grad Norm: 0.0191, LR: 0.000010
Epoch 11/50, Loss: 0.0433, Avg Grad Norm: 0.0026, LR: 0.000010
Epoch 12/50, Loss: 0.0433, Avg Grad Norm: 0.0027, LR: 0.000010
Epoch 13/50, Loss: 0.0433, Avg Grad Norm: 0.0011, LR: 0.000010
Epoch 14/50, Loss: 0.0434, Avg Grad Norm: 0.0086, LR: 0.000001
Epoch 15/50, Loss: 0.0431, Avg Grad Norm: 0.0129, LR: 0.000001
Epoch 16/50, Loss: 0.0432, Avg Grad Norm: 0.0038, LR: 0.000001
E